<a href="https://colab.research.google.com/github/jaehyun0220/Colab/blob/master/jigsaw_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 데이터 활용에 필요한 기본 패키지 로딩
import sys #access to system parameters 
print("Python version: {}". format(sys.version))

import pandas as pd
print("pandas version: {}". format(pd.__version__))

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import tensorflow as tf
print("tensorflow version: {}".format(tf.__version__))

import keras
print("keras version: {}".format(keras.__version__))

import os
#import io

Python version: 3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0]
pandas version: 0.24.2
scikit-learn version: 0.21.1
NumPy version: 1.16.3
tensorflow version: 1.13.1
keras version: 2.2.4


Using TensorFlow backend.


In [0]:
#데이터 전처리 관련 라이브러리 로드
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import ShuffleSplit

# Deep Learning Model 로드
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation 
from keras.wrappers.scikit_learn import KerasClassifier

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

#HyperParameter Tuning을 위한 라이브러리 로드
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#모델 평가를 위한 라이브러리 로드
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn import model_selection

#수학 & 통계 관련 라이브러리 로드
import scipy.stats as st
from collections import Counter

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Auth 인증 및 Google Drive 활용 Data load
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/gdrive', force_remount=True)

# Google Drive 내 Custom Class 경로 지정
import sys
sys.path.insert(0, '/gdrive/My Drive/CustomClasses')

Mounted at /gdrive


In [0]:
# from google.colab import files
# files.upload()
# !cp ./kaggle.json /root/.kaggle/kaggle.json
# !cp ./kaggle.json /gdrive/My\ Drive/kaggle/kaggle.json

In [5]:
# kaggle 실행을 위한 json file copy & 권한 부여
!cp /gdrive/My\ Drive/kaggle/kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json
!cat /root/.kaggle/kaggle.json
# !cat /gdrive/My\ Drive/CustomClasses/.kaggle/kaggle.json
# !kaggle competitions list

{"username":"claypark","key":"bb187008d01e8abfeaabdb973ebfa7a9"}

In [6]:
# !ls /gdrive/My\ Drive/kaggle
!ls /gdrive/My\ Drive/kaggle/jigsaw

sample_submission.csv  test.csv  train.csv


In [14]:
# Kaggle Competition Data Download - JIGSAW competition

# Directory 생성 후 데이터 다운로드
# !mkdir /gdrive/My\ Drive/kaggle/jigsaw  
# !kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification -p /gdrive/My\ Drive/kaggle/jigsaw 

# 파일 확인 후 각각 unzip 후 zip 파일 삭제
# !ls /gdrive/My\ Drive/kaggle/jigsaw
# !unzip /gdrive/My\ Drive/kaggle/jigsaw/train.csv.zip -d /gdrive/My\ Drive/kaggle/jigsaw
# !unzip /gdrive/My\ Drive/kaggle/jigsaw/test.csv.zip -d /gdrive/My\ Drive/kaggle/jigsaw
# !unzip /gdrive/My\ Drive/kaggle/jigsaw/sample_submission.csv.zip -d /gdrive/My\ Drive/kaggle/jigsaw
# !rm /gdrive/My\ Drive/kaggle/jigsaw/*.zip

!ls /gdrive/My\ Drive/kaggle/jigsaw

  0% 0.00/221k [00:00<?, ?B/s]
100% 221k/221k [00:00<00:00, 28.3MB/s]
 74% 9.00M/12.1M [00:00<00:00, 87.6MB/s]
100% 12.1M/12.1M [00:00<00:00, 77.1MB/s]
 99% 270M/273M [00:02<00:00, 122MB/s]
100% 273M/273M [00:02<00:00, 120MB/s]
sample_submission.csv	   test.csv	 train.csv
sample_submission.csv.zip  test.csv.zip  train.csv.zip


In [15]:
# !rm /gdrive/My\ Drive/kaggle/jigsaw/*.csv
# !ls /gdrive/My\ Drive/kaggle/jigsaw

sample_submission.csv.zip  test.csv.zip  train.csv.zip


# **EDA**

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud ,STOPWORDS
from PIL import Image
train = pd.read_csv('/gdrive/My Drive/kaggle/jigsaw/train.csv.zip', encoding='utf-8')
test = pd.read_csv('/gdrive/My Drive/kaggle/jigsaw/test.csv.zip', encoding='utf-8')
sub = pd.read_csv('/gdrive/My Drive/kaggle/jigsaw/sample_submission.csv.zip', encoding='utf-8')

In [21]:
train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [25]:
print(train.isnull().sum())
print("-"*50)
print("total train len is", len(train))
print("-"*50)
print(test.isnull().sum())
print("total test len is", len(test))

id                                           0
target                                       0
comment_text                                 0
severe_toxicity                              0
obscene                                      0
identity_attack                              0
insult                                       0
threat                                       0
asian                                  1399744
atheist                                1399744
bisexual                               1399744
black                                  1399744
buddhist                               1399744
christian                              1399744
female                                 1399744
heterosexual                           1399744
hindu                                  1399744
homosexual_gay_or_lesbian              1399744
intellectual_or_learning_disability    1399744
jewish                                 1399744
latino                                 1399744
male         